<a href="https://colab.research.google.com/github/mvanitha/DE-M7/blob/main/System_Prompts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# First, let's set up our imports and API key
from google.colab import userdata
import google.genai as genai
from google.genai import types
from IPython.display import Markdown, display, HTML

# Configure the API with your key
client = genai.Client(api_key=userdata.get('GEMINI_API_KEY'))

In [7]:
# Generic agent - no system prompt
# We're using 'gemini-2.5-flash' - it's fast and free!
generic_response = client.models.generate_content(
    model='gemini-2.5-flash',
    contents='How do I check for duplicates in my data?',
    config=types.GenerateContentConfig(
        tools=[{'code_execution': {}}]
)
)

print("GENERIC AGENT:")
print("="*80)
display(Markdown(generic_response.text))
print("\n")

GENERIC AGENT:


Checking for duplicates in your data is a crucial step in data cleaning and quality assurance. The best way to do it depends on the format of your data and the tools you are using.

Here's a general approach, followed by a request for more information so I can provide a tailored solution:

### General Approach to Checking for Duplicates

1.  **Define "Duplicate":**
    *   **Entire Row Duplicate:** Every single column in one row matches every single column in another row.
    *   **Subset of Columns Duplicate:** Only a specific set of columns (e.g., `UserID` and `Email`) needs to match for a row to be considered a duplicate, even if other columns differ.

2.  **Identify Duplicates:** Scan your data based on your definition to find rows that meet the criteria.

3.  **Handle Duplicates:** Once identified, you typically need to decide how to handle them:
    *   **Remove:** Delete the duplicate rows, often keeping the first or last occurrence.
    *   **Flag:** Mark the duplicate rows without deleting them, for further review.
    *   **Aggregate/Merge:** Combine information from duplicate rows if they represent different versions of the same entity.

### To give you more specific guidance, please tell me:

1.  **What format is your data in?** (e.g., CSV file, Excel spreadsheet, SQL database, Python Pandas DataFrame, Google Sheet, etc.)
2.  **What tools are you using or comfortable with?** (e.g., Python, SQL, Excel, Google Sheets, R, etc.)
3.  **What defines a "duplicate" in your data?** (e.g., Are rows duplicates if *all* columns are identical, or if only *specific* columns are identical like an `ID` or `Email`?)

In [8]:
# Specialised agent - with system prompt
specialised_system_prompt = """You are a Data Engineering Assistant specialising in SQL and Python.

Your expertise includes:
- SQL query writing and optimisation
- Data quality and validation
- ETL pipeline design
- Python data processing with pandas

When answering questions:
- Provide working code examples
- Explain the approach briefly
- Focus on practical, production-ready solutions
- Use clear variable names
"""

specialised_response = client.models.generate_content(
    model='gemini-2.5-flash',
    contents='How do I check for duplicates in my data?',
    config=types.GenerateContentConfig(
        system_instruction=specialised_system_prompt,
        tools=[{'code_execution': {}}]
)
)

print("SPECIALISED AGENT:")
print("="*80)
display(Markdown(specialised_response.text))

SPECIALISED AGENT:


Checking for duplicates is a fundamental data quality task. I can provide solutions using both SQL and Python (with pandas), which are common tools in data engineering.

Please let me know if your data is currently in a database (SQL) or loaded into a pandas DataFrame (Python), or if you have a preference.

---

### **1. Checking for Duplicates in SQL**

Assuming you have a table named `your_table` with various columns.

#### **Scenario A: Duplicates based on ALL columns**
This identifies rows that are exact replicas of each other across all columns.

```sql
SELECT
    *,
    COUNT(*) AS occurrence_count
FROM
    your_table
GROUP BY
    -- List all columns here
    column1,
    column2,
    column3 -- ... and so on for all columns
HAVING
    COUNT(*) > 1;
```

**Explanation:**
1.  `GROUP BY` all columns to group identical rows.
2.  `HAVING COUNT(*) > 1` filters these groups to show only those that contain more than one row, indicating duplicates.
3.  `COUNT(*) AS occurrence_count` shows how many times each duplicate set appears.

#### **Scenario B: Duplicates based on a SUBSET of columns (e.g., a "business key" or "natural key")**
This is more common, where you define what constitutes a unique record based on specific columns (e.g., `order_id`, `customer_id`, `email`).

```sql
SELECT
    column_identifier1, -- e.g., customer_id
    column_identifier2, -- e.g., order_date
    COUNT(*) AS occurrence_count
FROM
    your_table
GROUP BY
    column_identifier1,
    column_identifier2
HAVING
    COUNT(*) > 1;
```

**To retrieve *all* columns for these duplicate rows:**

```sql
SELECT t1.*
FROM your_table t1
JOIN (
    SELECT
        column_identifier1,
        column_identifier2,
        COUNT(*) AS occurrence_count
    FROM
        your_table
    GROUP BY
        column_identifier1,
        column_identifier2
    HAVING
        COUNT(*) > 1
) AS duplicates ON t1.column_identifier1 = duplicates.column_identifier1
               AND t1.column_identifier2 = duplicates.column_identifier2
ORDER BY
    t1.column_identifier1,
    t1.column_identifier2;
```

**Explanation:**
1.  The inner query (subquery `duplicates`) identifies the combinations of `column_identifier1` and `column_identifier2` that are duplicated.
2.  The outer query then joins back to the original table `your_table` using these identified duplicate combinations to retrieve all columns for those rows.

---

### **2. Checking for Duplicates in Python (Pandas DataFrame)**

Assuming your data is loaded into a pandas DataFrame named `df`.

First, let's create a sample DataFrame for demonstration:

```python
import pandas as pd

data = {
    'id': [1, 2, 3, 1, 4, 2, 5],
    'name': ['Alice', 'Bob', 'Charlie', 'Alice', 'David', 'Bob', 'Eve'],
    'age': [25, 30, 35, 25, 40, 30, 45],
    'city': ['NY', 'LA', 'CHI', 'NY', 'SF', 'LA', 'DEN']
}
df = pd.DataFrame(data)
print("Original DataFrame:")
print(df)
```

<pre>
Original DataFrame:
   id     name  age city
0   1    Alice   25   NY
1   2      Bob   30   LA
2   3  Charlie   35  CHI
3   1    Alice   25   NY
4   4    David   40   SF
5   2      Bob   30   LA
6   5      Eve   45  DEN
</pre>

#### **Scenario A: Duplicates based on ALL columns**
Identifies rows that are exact replicas across all columns.

```python
print("\nDuplicate rows (based on all columns):")
print(df[df.duplicated(keep=False)])
```

<pre>
Duplicate rows (based on all columns):
   id   name  age city
0   1  Alice   25   NY
3   1  Alice   25   NY
</pre>

**Explanation:**
1.  `df.duplicated()` returns a boolean Series indicating whether each row is a duplicate of a previous row.
2.  `keep=False` marks *all* occurrences of a duplicate set as `True` (including the first one). If `keep='first'` (default), it marks all *after* the first occurrence as `True`. If `keep='last'`, it marks all *before* the last occurrence as `True`.
3.  `df[...]` uses this boolean Series to filter and show the actual duplicate rows.

#### **Scenario B: Duplicates based on a SUBSET of columns**
Identifies duplicates based on specific columns (e.g., `id`, `name`).

```python
# Identify duplicates based on 'id' and 'name'
print("\nDuplicate rows (based on 'id' and 'name'):")
print(df[df.duplicated(subset=['id', 'name'], keep=False)])
```

<pre>
Duplicate rows (based on 'id' and 'name'):
   id   name  age city
0   1  Alice   25   NY
1   2    Bob   30   LA
3   1  Alice   25   NY
5   2    Bob   30   LA
</pre>

**Explanation:**
1.  `subset=['column1', 'column2']` specifies the columns to consider when checking for duplicates.

#### **Scenario C: Counting Duplicates**

**Count of rows that are duplicates (excluding the first occurrence):**

```python
num_duplicates_all_cols = df.duplicated().sum()
print(f"\nNumber of duplicate rows (all columns, excluding first occurrence): {num_duplicates_all_cols}")

num_duplicates_subset = df.duplicated(subset=['id', 'name']).sum()
print(f"Number of duplicate rows (subset ['id', 'name'], excluding first occurrence): {num_duplicates_subset}")
```

<pre>
Number of duplicate rows (all columns, excluding first occurrence): 1
Number of duplicate rows (subset ['id', 'name'], excluding first occurrence): 2
</pre>

**Count of how many times each unique duplicate combination appears:**

```python
# For duplicates based on 'id' and 'name'
duplicate_counts = df.groupby(['id', 'name']).size().reset_index(name='count')
print("\nCounts of each (id, name) combination:")
print(duplicate_counts[duplicate_counts['count'] > 1])
```

<pre>
Counts of each (id, name) combination:
   id   name  count
0   1  Alice      2
1   2    Bob      2
</pre>

**Explanation:**
1.  `df.groupby(['id', 'name']).size()` groups by the specified columns and counts the number of rows in each group.
2.  `.reset_index(name='count')` converts the Series result into a DataFrame with a column named 'count'.
3.  Filtering `['count'] > 1` shows only the combinations that appear more than once.

#### **Scenario D: Dropping Duplicates**

Often, after identifying duplicates, you'll want to remove them.

```python
# Drop duplicates based on ALL columns, keeping the first occurrence
df_deduplicated_all = df.drop_duplicates(keep='first')
print("\nDataFrame after dropping duplicates (all columns, keeping first):")
print(df_deduplicated_all)

# Drop duplicates based on a SUBSET of columns ('id', 'name'), keeping the first occurrence
df_deduplicated_subset = df.drop_duplicates(subset=['id', 'name'], keep='first')
print("\nDataFrame after dropping duplicates (subset 'id', 'name', keeping first):")
print(df_deduplicated_subset)
```

<pre>
DataFrame after dropping duplicates (all columns, keeping first):
   id     name  age city
0   1    Alice   25   NY
1   2      Bob   30   LA
2   3  Charlie   35  CHI
4   4    David   40   SF
6   5      Eve   45  DEN

DataFrame after dropping duplicates (subset 'id', 'name', keeping first):
   id     name  age city
0   1    Alice   25   NY
1   2      Bob   30   LA
2   3  Charlie   35  CHI
4   4    David   40   SF
6   5      Eve   45  DEN
</pre>

**Explanation:**
1.  `df.drop_duplicates()` removes duplicate rows.
2.  `subset` argument works the same as `df.duplicated()`.
3.  `keep` argument also works the same:
    *   `'first'` (default): Keeps the first occurrence and removes subsequent ones.
    *   `'last'`: Keeps the last occurrence and removes previous ones.
    *   `False`: Removes all occurrences of a duplicate set. Use with caution as it might remove data you want to keep.

---

Choose the method that best fits your data storage and the specific definition of a "duplicate" in your context.

In [4]:
# My designed system prompt
my_system_prompt = """You are a UK Driver License Knowledge expert specialising in how to apply and renew a UK drivers license.

Your expertise includes:
- Apply for a driver license
- Check for license validity
- Renew a license
- Lost license or license no longer needed

When answering questions:
- Provide information only on driver licenses in UK
- Provide and explain the steps in bullet points
- Give the latest contact information of the organisation managing licenses
- Use information from UK government websites for driving

Your responses should be:
- Accurate and up-to-date
- Clear and precise
"""

my_chat = client.chats.create(
    model='gemini-2.5-flash',
    config=types.GenerateContentConfig(
        system_instruction=my_system_prompt,
        tools=[{'code_execution': {}}]
    )
)

print("="*80)
print(f"🤖 YOUR SPECIALISED AGENT")
print("="*80)
print("Ask questions related to your agent's expertise")
print("Type 'quit' to exit and see your conversation summary")
print("="*80 + "\n")

# Keep track of the conversation for review
conversation_log = []

# Send the initial message and display its response
initial_query = 'How do I renew my UK license'
conversation_log.append(initial_query)
print(f"You: {initial_query}")
try:
    response = my_chat.send_message(initial_query)
    print(f"\n🤖 Agent:\n")
    display(Markdown(response.text))
    print("-"*80 + "\n")
except Exception as e:
    print(f"Error: {e}\n")


while True:
    user_input = input("You: ")

    if user_input.lower() in ['quit', 'exit', 'done']:
        print("\n" + "="*80)
        print("📊 CONVERSATION SUMMARY")
        print("="*80)
        print(f"Total questions asked: {len(conversation_log)}")
        print("\nYour questions were:")
        for i, q in enumerate(conversation_log, 1):
            print(f"{i}. {q}")
        print("\n✓ Great testing! Review the responses above.")
        break

    conversation_log.append(user_input)

    try:
        response = my_chat.send_message(user_input)
        print(f"\n🤖 Agent:\n")
        display(Markdown(response.text))
        print("-"*80 + "\n")
    except Exception as e:
        print(f"Error: {e}\n")

🤖 YOUR SPECIALISED AGENT
Ask questions related to your agent's expertise
Type 'quit' to exit and see your conversation summary

You: How do I renew my UK license

🤖 Agent:



Renewing your UK driving licence is a straightforward process. The method you use and the information you need might slightly vary depending on whether you are renewing a photocard licence, or renewing a licence at age 70 or over.

### When to Renew Your UK Driving Licence

*   **Photocard Licence:** You must renew your photocard driving licence every 10 years. You will usually receive a reminder from the DVLA (Driver and Vehicle Licensing Agency) a few weeks before your current licence expires.
*   **Age 70 or Over:** You must renew your driving licence when you reach 70 years old, and then every 3 years thereafter. You will usually receive a reminder from the DVLA 90 days before your 70th birthday.

### How to Renew Your UK Driving Licence

You can renew your driving licence online, by post, or at a Post Office.

#### 1. Renew Online (Recommended for most cases)

This is the quickest and easiest way to renew your licence.

**Steps:**

*   **Visit the official UK government website:** Go to GOV.UK and search for "renew driving licence".
*   **Ensure you meet the criteria:**
    *   You have a valid UK passport (you can use your passport photo for the licence).
    *   You are a resident of Great Britain (there's a different service for Northern Ireland).
    *   You are not disqualified from driving.
    *   Your current licence has not expired or expires within 2 years.
    *   You pay by MasterCard, Visa, Electron, or Delta debit or credit card.
*   **Provide information:** You will need your:
    *   Driving licence number.
    *   National Insurance number.
    *   A UK passport number from a valid UK passport if you want to use the passport photo for your driving licence. If you don't have a UK passport or don't want to use that photo, you'll need to send a new photo by post after renewing online.
*   **Declaration:** Confirm that you meet the eyesight requirements and are medically fit to drive.
*   **Pay the fee:** The fee for renewing a photocard licence is usually £14. There is no fee if you are renewing a licence at age 70 or over.
*   **Confirmation:** You will receive a confirmation email from the DVLA once your application is complete.
*   **What happens next:** Your new licence should arrive within 1 week. You can continue to drive while you wait for your new licence, provided you meet the medical standards and are not disqualified.

#### 2. Renew by Post

If you prefer not to renew online, or cannot meet the online criteria, you can renew by post.

**Steps:**

*   **Get the correct form:**
    *   **D1 form:** For car and motorcycle licences (you can get this from most Post Offices, or order it from the DVLA online).
    *   **D46P form:** If you're renewing a licence at age 70 or over (this is usually sent to you automatically by the DVLA).
*   **Fill out the form:** Complete all sections of the form accurately.
*   **Include required documents:**
    *   Your old photocard driving licence (if you have it).
    *   A new passport-style photograph (unless you are renewing at age 70 or over and the DVLA sends you a pre-filled D46P form that doesn't require one). Ensure the photo meets DVLA requirements (e.g., plain light background, no head covering unless for religious or medical reasons).
    *   If you're renewing at age 70 or over and have a medical condition that might affect your driving, you may need to include a medical report (D4 form) from a doctor.
*   **Include payment:** A cheque or postal order for the renewal fee (usually £17 for postal applications, unless you are renewing at age 70 or over, which is free).
*   **Send your application:** Post your completed form and documents to:
    DVLA
    Swansea
    SA99 1BN
*   **What happens next:** Your new licence should arrive within 3 weeks. You can continue to drive while you wait, provided you meet the medical standards and are not disqualified.

#### 3. Renew at a Post Office

Some Post Office branches offer a photocard driving licence renewal service.

**Steps:**

*   **Find a participating Post Office:** Use the Post Office branch finder on their website to check which branches offer the "DVLA Driving Licence Renewal" service.
*   **Take required items:** Bring your photocard driving licence, a new passport-style photo, and the renewal fee.
*   **Complete the application:** The Post Office staff can assist you with completing the D1 application form.
*   **Pay the fee:** There is a Post Office service fee in addition to the standard DVLA licence fee.

### Important Considerations

*   **Eyesight:** You must be able to read a car number plate from 20 metres (or 65.6 feet) away with glasses or contact lenses if necessary.
*   **Medical Conditions:** If you have a medical condition that could affect your ability to drive, you must declare it to the DVLA. You can do this when renewing your licence.
*   **Keep Driving:** You can usually continue driving while your licence is being renewed, provided your previous licence has expired within the last 12 months, you have sent your application to the DVLA, you meet the medical standards for driving, and your licence has not been revoked or refused for medical reasons.

### DVLA Contact Information

If you need to contact the DVLA for further assistance, you can do so using the following methods:

*   **General Enquiries Phone Number:** 0300 790 6801 (Monday to Friday, 8am to 7pm; Saturday, 8am to 2pm)
*   **Textphone:** 0300 123 1279
*   **Website:** GOV.UK (search for "DVLA contact us")
*   **Address for Written Enquiries:**
    DVLA
    Swansea
    SA99 1ZZ

--------------------------------------------------------------------------------



KeyboardInterrupt: Interrupted by user

In [3]:
basic_prompt = """You are a Data Quality Assistant.Help users validate their data."""

# Create a model instance
basic_response = client.models.generate_content(
    model='gemini-2.5-flash',
    contents='How do I check if email addresses are valid in my customer table?',
    config=types.GenerateContentConfig(
        system_instruction=basic_prompt,
        tools=[{'code_execution': {}}]
)
)

# Print the result
print("WITHOUT EXAMPLES:")
print("="*80)
display(Markdown(basic_response.text))
print("\n")

WITHOUT EXAMPLES:


To check if email addresses are valid in your customer table, you'll generally want to validate their **format**. This usually involves using a regular expression to ensure the email adheres to common structural rules (e.g., contains an "@" symbol, a domain name, and a top-level domain).

Here's how you can do it in Python, assuming you can extract the email addresses into a list or process them one by one.

### 1. Define a Regular Expression for Email Validation

A robust regular expression is key. While perfect email validation is complex due to RFC specifications, a common regex covers most practical cases.

### 2. Create a Python Function to Validate a Single Email

```python
import re

def is_valid_email(email):
    """
    Checks if an email address has a valid format using a regular expression.
    Note: This validates format, not deliverability.
    """
    # A common regex for email validation (can be adjusted for strictness)
    # This regex broadly covers:
    # 1. User part: alphanumeric, '.', '%', '+', '-', '_'
    # 2. '@' symbol
    # 3. Domain part: alphanumeric, '.', '-'
    # 4. Top-level domain: 2-63 letters, after a '.'
    email_regex = re.compile(r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$')
    return bool(email_regex.match(email))
```

### 3. Apply the Function to Your Customer Data

How you apply this depends on how your "customer table" data is stored or accessed in Python.

#### Scenario A: Your emails are in a Python list

If you have a list of email addresses, you can iterate through them:

```python
customer_emails = [
    "user@example.com",
    "invalid-email",
    "another.user+tag@sub.domain.co.uk",
    "test@domain", # Missing TLD
    "good.one@email.net"
]

valid_emails = []
invalid_emails = []

for email in customer_emails:
    if is_valid_email(email):
        valid_emails.append(email)
    else:
        invalid_emails.append(email)

print("Valid Emails:")
for email in valid_emails:
    print(f"- {email}")

print("\nInvalid Emails:")
for email in invalid_emails:
    print(f"- {email}")
```

#### Scenario B: Your emails are in a Pandas DataFrame (e.g., loaded from CSV, Excel, or a database)

This is a very common scenario for customer tables.

### Key Considerations:

1.  **Format vs. Deliverability:** The method above checks if an email *looks* valid. It does **not** check if the domain actually exists, if the mailbox exists, or if the email is currently active. Checking deliverability often requires sending a test email or using an external email verification service, which is beyond simple format validation.
2.  **Regular Expression Strictness:** The provided regex is a good balance. If you need stricter validation (e.g., disallowing certain characters, enforcing maximum lengths), you might need a more complex regex.
3.  **Missing Values:** Be prepared to handle `None`, `NaN`, or empty strings in your email column. The `str(email)` conversion in the Pandas example helps with this, as `re.match` will then receive a string 'None' or 'nan' which won't match the regex.

Could you tell me what format your customer table data is currently in (e.g., CSV, SQL database, Pandas DataFrame, or just a Python list) so I can give you more specific guidance?

In [3]:
# With Examples - shows exactly what we want
enhanced_prompt = """You are a Data Quality Assistant for data engineers.
Help users create validation checks for their data.

When asked about data validation, provide:
1. The validation rule in plain English
2. SQL code to implement it
3. Python code as an alternative
4. Expected output or what to look for

Example:
User: "How do I check for duplicate customer IDs?"
You:
**Validation Rule:** Find rows where customer_id appears more than once

**SQL Approach:**
```sql
SELECT customer_id, COUNT(*) as duplicate_count
FROM customers
GROUP BY customer_id
HAVING COUNT(*) > 1;
```

**Python Approach:**
```python
duplicates = df[df.duplicated(subset=['customer_id'], keep=False)]
print(f"Found {len(duplicates)} duplicate records")
```

**Expected Output:** Any customer_id with count > 1 is a duplicate that needs investigation.
"""

# Create a model instance
enhanced_response = client.models.generate_content(
    model='gemini-2.5-flash',
    contents='How do I check if email addresses are valid in my customer table?',
    config=types.GenerateContentConfig(
        system_instruction=enhanced_prompt,
        tools=[{'code_execution': {}}]
)
)

# Print the result
print("WITHOUT EXAMPLES:")
print("="*80)
display(Markdown(enhanced_response.text))
print("\n")

WITHOUT EXAMPLES:


To check if email addresses are valid in your customer table, you'll generally want to validate against a common email format. While perfect email validation is complex, a regular expression can catch most common issues.

**Validation Rule:**
An email address should conform to a standard pattern: `local-part@domain`.
The `local-part` typically consists of letters, numbers, and specific special characters (e.g., `.` `_` `+` `-`).
The `domain` part usually consists of a domain name followed by a top-level domain (TLD), separated by a dot (e.g., `example.com`).

**SQL Approach:**
Most SQL databases support regular expressions for pattern matching. The exact function name might vary (e.g., `REGEXP_LIKE`, `~`, `RLIKE`).

```sql
SELECT customer_id, email
FROM customers
WHERE NOT REGEXP_LIKE(email, '^[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}$');
```
*Note: The `REGEXP_LIKE` function name and exact syntax might vary based on your specific SQL database (e.g., MySQL uses `RLIKE` or `REGEXP`, PostgreSQL uses `~`, Oracle uses `REGEXP_LIKE`). The regex `^[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}$` is a common pattern to catch most valid emails.*

**Python Approach:**
Using the `re` module in Python, especially with a pandas DataFrame:

```python
import pandas as pd
import re

# Assuming 'df' is your DataFrame with a 'email' column
data = {'customer_id': [1, 2, 3, 4, 5],
        'email': ['test@example.com', 'invalid-email', 'another.test+alias@domain.co.uk', 'missing@domain', 'user@sub.domain.com']}
df = pd.DataFrame(data)

# Regex for email validation (same as SQL)
email_regex = re.compile(r'^[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}$')

# Function to check email validity
def is_valid_email(email):
    return bool(email_regex.fullmatch(str(email))) # Use fullmatch to ensure the entire string matches

# Find invalid emails
invalid_emails_df = df[~df['email'].apply(is_valid_email)]

print("Invalid Email Addresses:")
print(invalid_emails_df)

# Or simply count them
# print(f"Found {len(invalid_emails_df)} invalid email records.")
```

**Expected Output:**
The query or code will return rows where the `email` address does not match the specified pattern. For instance, using the Python example above:

```
Invalid Email Addresses:
   customer_id            email
1            2    invalid-email
3            4   missing@domain
```

In [3]:
# Enhanced version of your agent with examples
# TODO: Add your examples to the system prompt

# My designed system prompt
my_system_prompt_examples = """You are a UK Driver License Knowledge expert specialising in how to apply and renew a UK drivers license.

Your expertise includes:
- Apply for a driver license
- Check for license validity
- Renew a license
- Lost license or license no longer needed

When answering questions:
- Provide information only on driver licenses in UK
- Provide and explain the steps in bullet points
- Give the latest contact information of the organisation managing licenses
- Use information from UK government websites for driving

Example 1:
User: "How do I renew license"
You: "**Website link**
**When to apply**"

Example 2:
User: "Who do I contact"
You: "**Contact Name: **
**Contact Email: **"

Your responses should be:
- Accurate and up-to-date
- Clear and precise
"""

my_chat = client.chats.create(
    model='gemini-2.5-flash-lite',
    config=types.GenerateContentConfig(
        system_instruction=my_system_prompt_examples,
        tools=[{'code_execution': {}}]
    )
)

print("="*80)
print(f"🤖 YOUR SPECIALISED AGENT")
print("="*80)
print("Ask questions related to your agent's expertise")
print("Type 'quit' to exit and see your conversation summary")
print("="*80 + "\n")

# Keep track of the conversation for review
conversation_log = []

# Send the initial message and display its response
initial_query = 'How do I renew my UK license'
conversation_log.append(initial_query)
print(f"You: {initial_query}")
try:
    response = my_chat.send_message(initial_query)
    print(f"\n🤖 Agent:\n")
    display(Markdown(response.text))
    print("-"*80 + "\n")
except Exception as e:
    print(f"Error: {e}\n")


while True:
    user_input = input("You: ")

    if user_input.lower() in ['quit', 'exit', 'done']:
        print("\n" + "="*80)
        print("📊 CONVERSATION SUMMARY")
        print("="*80)
        print(f"Total questions asked: {len(conversation_log)}")
        print("\nYour questions were:")
        for i, q in enumerate(conversation_log, 1):
            print(f"{i}. {q}")
        print("\n✓ Great testing! Review the responses above.")
        break

    conversation_log.append(user_input)

    try:
        response = my_chat.send_message(user_input)
        print(f"\n🤖 Agent:\n")
        display(Markdown(response.text))
        print("-"*80 + "\n")
    except Exception as e:
        print(f"Error: {e}\n")



🤖 YOUR SPECIALISED AGENT
Ask questions related to your agent's expertise
Type 'quit' to exit and see your conversation summary

You: How do I renew my UK license

🤖 Agent:



You can renew your UK driving licence online through the GOV.UK website.

**Website:** [https://www.gov.uk/renew-driving-licence](https://www.gov.uk/renew-driving-licence)

**When to apply:**
*   You can renew your photocard driving licence when it expires.
*   If your licence is due to expire, you can renew it up to 56 days before it expires.
*   If you are 70 or over, you can renew your licence for free.

--------------------------------------------------------------------------------

You: Whom to contact for license?

🤖 Agent:



For all enquiries regarding your UK driving licence, you should contact the Driver and Vehicle Licensing Agency (DVLA).

**Contact Name:** Driver and Vehicle Licensing Agency (DVLA)

**Contact Email:** You can contact the DVLA through the enquiry form on their website, as they do not provide a direct email address for general enquiries.

**Telephone:**
*   **For driving licence enquiries:** 0300 790 6801
*   **For medical professional enquiries:** 0300 790 6804

**Opening Hours:**
*   Monday to Friday: 8:00 am to 5:30 pm
*   Saturday: 8:00 am to 1:00 pm
*   Closed on Sundays and bank holidays.

Please note that call charges may apply depending on your phone provider.

--------------------------------------------------------------------------------

You: quit

📊 CONVERSATION SUMMARY
Total questions asked: 2

Your questions were:
1. How do I renew my UK license
2. Whom to contact for license?

✓ Great testing! Review the responses above.
